In [1]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np
import oxyba as ox; from importlib import reload; reload(ox);
from time import perf_counter, process_time

### Load Demo Dataset

In [2]:
from sklearn.datasets import load_boston
tmp = load_boston()
num_obs = len(tmp.target);
y = tmp.target
X = np.c_[ np.ones(shape=(num_obs,1)), tmp.data[:,[5,12]] ];

### How to estimate? (What is OLS?)
Let $y \in \mathbb{R}^{n \times 1}$ the dependent variable, 
$X \in \mathbb{R}^{n \times m}$ the indendent variables (including the intercept), 
and $\beta \in \mathbb{R}^{m \times 1}$ the regression coefficients.

The residuals or errors $\epsilon \in \mathbb{R}^{n \times 1}$ are the difference between $y$ and the prediction $X\beta$ 

$$
\epsilon = y - X \beta
$$

and the **Mean Squared Error** is

$$
MSE(\beta) = \frac{1}{n} \epsilon^T \epsilon 
$$

The **gradient** of the MSE is

$$
\nabla \, MSE(\beta) = \frac{2}{n} (X^T X\beta - X^T y)
$$

The gradient would be $\nabla MSE(\hat{\beta}) = 0$ for the **optimum** solution $\hat{\beta}$

$$
0 = X^T X\hat{\beta} - X^T y \\
\Leftrightarrow \hat{\beta} = (X^T X)^{-1} - (X^T y)
$$

The Ordinary Least Square (OLS) method is basically a) setting the gradient of the MSE to zero, and b) solving after $\beta$.

### Solving by LU decomposition
The oxyba package has a function `linreg_ols_lu`. 
In this chapter we will compare different implementations in python.

At its core `linreg_ols_lu` tries to solve $Ax=b$ wheras $A=X^T X$, $x=\beta$ and $b=X^T y$

$$
\min_x Ax - b
$$

wheras LU decomposition is applied to $A = LU = PA$ at some point in the program.

The second test implementation (I spare you from the first one) appies scipy's [lu_factor](https://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.linalg.lu_factor.html#scipy.linalg.lu_factor) and [lu_solve](https://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.linalg.lu_solve.html#scipy.linalg.lu_solve) after another.
`func2_pretty` is the more readable and slower version of `func2_faster`.

In [3]:
def func2_pretty(y,X):
    from numpy import dot;
    from scipy.linalg import lu_factor, lu_solve;
    A = dot(X.T, X);
    b = dot(X.T, y);
    lu,piv = lu_factor(A)
    return lu_solve((lu,piv),b)

In [4]:
def func2_faster(y,X):
    import numpy as np;
    import scipy;
    return scipy.linalg.lu_solve(scipy.linalg.lu_factor(np.dot(X.T,X)), np.dot(X.T,y));

The third implementation is used for `linreg_ols_lu`. 
It applies Numpy's [solve](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.solve.html)
what in turn is based on LAPACK's [_gesv](http://www.netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html#ga5ee879032a8365897c3ba91e3dc8d512)
that applies LU decomposition. 
It is not really obvious that `numpy.linalg.solve` is Linear Regression by LU decomposition if you don't read the manual. 
`func3_pretty` is the more readable and slower version of `func3_faster`.

In [5]:
def func3_pretty(y,X):
    from numpy import dot;
    from numpy.linalg import solve;
    return solve(dot(X.T,X), dot(X.T,y));

In [6]:
def func3_faster(y,X):
    import numpy as np;
    return np.linalg.solve(np.dot(X.T,X), np.dot(X.T,y));

Surprise, surprise. Numpy's [inv](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv) is actually a wrapper for `numpy.linalg.solve`.

In [7]:
def func4_pretty(y,X):
    from numpy import dot
    from numpy.linalg import inv
    x2inv = inv(dot(X.T, X));
    return dot(x2inv, dot(X.T,y)) 

In [8]:
def func4_faster(y,X):
    import numpy as np
    return np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T,y)) 

### Benchmark

In [9]:
trials = 50000
funcnames = [func2_pretty, func2_faster, 
             func3_pretty, func3_faster, 
             func4_pretty, func4_faster, 
             ox.linreg_ols_lu]

Let's conduct a quick check if results are correct

In [10]:
for func in funcnames:
    beta = func(y,X);
    print(func.__name__, beta);

func2_pretty [-1.35827281  5.09478798 -0.64235833]
func2_faster [-1.35827281  5.09478798 -0.64235833]
func3_pretty [-1.35827281  5.09478798 -0.64235833]
func3_faster [-1.35827281  5.09478798 -0.64235833]
func4_pretty [-1.35827281  5.09478798 -0.64235833]
func4_faster [-1.35827281  5.09478798 -0.64235833]
linreg_ols_lu [-1.35827281  5.09478798 -0.64235833]


Looks Ok. Proceed with the benchmark

In [11]:
print('{0:6s} {1:6s} {2:s}'.format('Clock', 'CPU', 'function name'))
for func in funcnames:
    sh,sc = perf_counter(), process_time();
    for i in range(trials):
        beta = func(y,X); 
        if beta is None: print('error solving')
        beta = None;
    eh,ec = perf_counter(), process_time()
    print('{0:.4f} {1:.4f} {2:s}'.format(eh-sh, ec-sc, func.__name__))

Clock  CPU    function name
5.0989 4.9988 func2_pretty
4.7060 4.6117 func2_faster
2.0440 2.0014 func3_pretty
1.7836 1.7566 func3_faster
1.9170 1.8274 func4_pretty
1.8330 1.8001 func4_faster
2.1270 2.0063 linreg_ols_lu


Scipy's `lu_factor` and `lu_solve` (`func2...`) are likely to be slower than Numpy's `solve` (`func3...`) due to the overhead by calling external C functions (LAPACK) twice.

`func4_...` (Numpy's `inv`) is as fast as `func3_...` (Numpy's `solve`) 
but I prefer `func3_faster` for my wrapper function `linreg_ols_lu`.

`linreg_ols_lu` is a little it slower than `func3_faster` due to additional error checking.